### How much do WWLLN cloud-to-ground lightning strikes vary from year to year?

from wildfires.data import WWLLN

In [ ]:
from wildfires.utils import get_land_mask

In [ ]:
for title, mask in zip(("WWLLN", "WWLLN over Land"), (None, ~get_land_mask(n_lon=720))):
    if mask is not None:
        wwlln = WWLLN().apply_masks(mask)
    else:
        wwlln = WWLLN()

    # Compute climatology
    wwlln_clim = wwlln.get_climatology_dataset(wwlln.min_time, wwlln.max_time)

    # Compute deviations of the individual years from the climatology
    n_years = wwlln.cube.shape[0] // 12

    aggs = []
    for i in range(n_years):
        yearly_data = wwlln.cube.data[12 * i : 12 * (i + 1)]
        diff = yearly_data - wwlln_clim.cube.data
        aggs.append(np.mean(np.abs(diff)))

    plt.figure()
    plt.title(title)
    plt.plot(np.arange(n_years) + wwlln.min_time.year, aggs)
    plt.xlabel("Year")
    plt.ylabel("<|diff|>")

    wwlln_mean = wwlln.get_mean_dataset()
    wwlln_mean, wwlln_mean.cube

    from wildfires.analysis import cube_plotting

    for i in range(n_years):
        yearly_data = wwlln.cube.data[12 * i : 12 * (i + 1)]
        diff = (
            np.mean(100 * (yearly_data - wwlln_clim.cube.data), axis=0)
            / wwlln_mean.cube.data
        )
        cube_plotting(
            diff,
            title=f"{title}, {i + wwlln.min_time.year}",
            colorbar_kwargs=dict(label="<(yearly - clim) / mean> (%)"),
            fig=plt.figure(figsize=(7, 3.2), dpi=120),
            vmin_vmax_percentiles=(5, 95),
            cmap="bwr",
            cmap_midpoint=0,
        )

    from wildfires.analysis import cube_plotting

    for i in range(n_years):
        yearly_data = wwlln.cube.data[12 * i : 12 * (i + 1)]
        diff = (
            np.mean(100 * np.abs(yearly_data - wwlln_clim.cube.data), axis=0)
            / wwlln_mean.cube.data
        )
        cube_plotting(
            diff,
            title=f"{title}, {i + wwlln.min_time.year}",
            colorbar_kwargs=dict(label="<|yearly - clim| / mean> (%)"),
            fig=plt.figure(figsize=(7, 3.2), dpi=120),
            vmin_vmax_percentiles=(2, 85),
        )